In [ ]:
import numpy as np
np.random.seed(0)
import random
random.seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [ ]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
fashion_mnist = datasets.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
print(f'Train images type: {type(train_images)}\nTrain labels type: {type(train_labels)}')

In [ ]:
print(f'Train images: {len(train_images)}\nTrain labels: {len(train_labels)}')
print(f'Train images shape: {train_images[0].shape}')

In [ ]:
print(train_images[0])

In [ ]:
plt.imshow(train_images[0], cmap='gray')

In [ ]:
train_labels[0]

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images = train_images.reshape((train_images.shape[0], 28,28,1))
test_images = test_images.reshape((test_images.shape[0], 28,28,1))

In [ ]:
print(f'Train images: {len(train_images)}\nTrain labels: {len(train_labels)}')
print(f'Train images shape: {train_images[0].shape}')

In [ ]:
print(f'Test images: {len(test_images)}\nTest labels: {len(test_labels)}')
print(f'Test images shape: {test_images[0].shape}')

In [ ]:
model = models.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.compile(optimizer = 'adam',
             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, epochs=5,
                   validation_data=(test_images, test_labels))

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test loss= {test_loss}\nTest accuracy = {test_acc}')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

In [ ]:
model.save('trained_fashion_mnist_model.h5')